In [1]:
from astroquery.vizier import Vizier
from astroquery.sdss import SDSS
import astropy.units as u
from astropy import coordinates as coords
from astropy.table import Table, hstack, vstack
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
from synphot import SourceSpectrum
from synphot.models import Empirical1D
from synphot import units
import random
import os.path
from collections import Counter
from specutils import Spectrum1D, SpectralRegion
from specutils.analysis import snr_derived


In [3]:
# 164 total subclasses, 58 hot star subclasses:
hotClass = [
'A0',
'A0 (19510)',
'A0IVn (25642)',
'A0p',
'A1Iae (12953)',
'A1III (225180)',
'A1m (78209)',
'A1V (95418)',
'A2Ia (14489)',
'A2II (39866)',
'A3Iae (223385)',
'A4 (G_165-39)',
'A4p (G_37-26)',
'A4V (136729)',
'A4V (97603)',
'A5Ia (17378)',
'A5II (34578)',
'A6IV (28527)',
'A8V (155514)',
'A9V (154660)',
'Am (78362)',
'B0.5Iae (185859)',
'B0.5Ibe... (187459)',
'B0IVe (5394)',
'B1Ve (212571)',
'B2.5V (175426)',
'B2.5Ve (187811)',
'B2Iaevar (41117)',
'B2III (35468)',
'B2IV-V (176819)',
'B2Ve (164284)',
'B2Vne (202904)',
'B2Vne (58343)',
'B3Ib/II (51309)',
'B3II (175156)',
'B3V (29763)',
'B3Ve (25940)',
'B5 (338529)',
'B5Ib (191243)',
'B5III (209419)',
'B5V (173087)',
'B6',
'B6IIIpe (109387)',
'B6IV (174959)',
'B7IVe (209409)',
'B8Ib (208501)',
'B8III (220575)',
'B8IV (171301)',
'B9',
'B9 (105262)',
'B9.5V+... (37269)',
'B9III (15318)',
'B9Vn (177756)',
'O',
'O8/O9 (167771)',
'O8e (188001)',
'O9.5Iae (30614)',
'OB'
]


In [10]:
hotStars = Table()
for x in range(len(hotClass)):
    query = "select specobjid, ra, dec, plate, mjd, fiberid, class, subclass, z from specObj where class='STAR' and subclass = '"+hotClass[x]+"'"
    res = SDSS.query_sql(query, data_release=17)
    hotStars = vstack([hotStars, res])


In [13]:
hotStars.write('tables/hotstars.ecsv')


In [14]:
hotStars=Table.read('tables/hotstars.ecsv')


In [15]:
# Pick 1000 random stars
hotStars_1k = hotStars[random.sample(range(len(hotStars)), 1000)]


In [17]:
hotStars_1k.write('tables/hotstars_1k.ecsv')


In [2]:
hotStars_1k=Table.read('tables/hotstars_1k.ecsv')
hotStars_1k


specobjid,ra,dec,plate,mjd,fiberid,class,subclass,z
float64,float64,float64,int32,int32,int32,str4,str19,float64
3.7312441944093245e+18,224.98038,-0.61224314,3314,54970,43,STAR,A0,0.0006146495
6.544868909614651e+18,141.37604,45.611524,5813,56363,46,STAR,A1V (95418),0.0003341742
7.646035853253564e+18,235.33571,56.732179,6791,56429,180,STAR,A2II (39866),-0.0009936119
9.319199541684361e+18,119.06644,43.124837,8277,57073,458,STAR,A4p (G_37-26),0.000177478
6.805186529247648e+18,244.46258,41.939843,6044,56090,900,STAR,A4V (97603),-0.0001727998
7.076336544668473e+18,10.994135,22.565733,6285,56248,202,STAR,A8V (155514),-0.0002458861
2.4871585894965187e+18,248.24663,14.754369,2209,53907,166,STAR,A0,-0.0001720352
6.811749514137065e+18,235.41106,41.823698,6050,56089,200,STAR,A9V (154660),-0.0008251562
3.6445908569309594e+18,196.41757,59.455818,3237,54883,192,STAR,A0,0.0001093311


In [3]:
# Download spectrum files:
import os.path
error = []
for l in range(len(hotStars_1k)):
        plate = hotStars_1k[l]['plate']
        mjd = hotStars_1k[l]['mjd']
        fiber = hotStars_1k[l]['fiberid']
        filename = 'spec-'+str(plate).zfill(4)+'-'+str(mjd)+'-'+str(fiber).zfill(4)+'.fits'
        if not os.path.exists('SDSSspec/'+filename):
            try:
                spec = SDSS.get_spectra(plate=plate, mjd=mjd, fiberID=fiber, data_release=17)
                spec[0].writeto('SDSSspec/'+filename)
            except:
                csv_row = str(plate).zfill(4)+', '+str(mjd)+', '+str(fiber).zfill(4)
                error.append(csv_row)

with open('SDSSspec/error.txt', 'w') as f:
    for element in error:
        f.write(element + "\n")


C:\Users\AntonioOrtega\AppData\Local\Programs\Python\Python39\lib\site-packages\astroquery\sdss\core.py:867: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  arr = np.atleast_1d(np.genfromtxt(io.BytesIO(response.content),
MC1TRCT =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Top                       [astropy.io.fits.card]
